In [1]:
%load_ext autoreload
%autoreload 2

from common import add_workspace_to_path

add_workspace_to_path()

In [2]:
from shared.secrets import get_secret, SecretKeys
from infrastructure.google.GoogleSecret import GoogleSecret

In [3]:
credentials_model = get_secret(SecretKeys.GoogleCredentials, GoogleSecret)

In [4]:
from google.oauth2.credentials import Credentials

credentials = Credentials.from_authorized_user_info(credentials_model.credentials.model_dump())

In [14]:
from datetime import datetime, timezone
from googleapiclient.discovery import build
from dataclasses import dataclass
from typing import Optional, Dict, Any
from datetime import datetime

@dataclass
class TaskList:
    kind: str
    id: str
    etag: str
    title: str
    updated: datetime
    selfLink: str

@dataclass
class Task:
    kind: str
    id: str
    etag: str
    title: str
    updated: datetime
    selfLink: str
    position: str
    notes: str
    status: str
    due: datetime
    links: list[Dict[str, Any]]
    webViewLink: str

In [15]:
from enum import Enum

class TaskListIds(Enum):
    Default = "MDY0Mzc2NjgyMDc4MTc0Nzg3Mjk6MDow"


In [ ]:

from shared.dates import get_end_of_day

class TaskService:
    def __init__(self, credentials):
        self.credentials = credentials
        self.service = build(
            "tasks", "v1", credentials=credentials, cache_discovery=False
        )

    def fetch_tasklists(self) -> list[TaskList]:
        tasklists_response = self.service.tasklists().list().execute()
        tasklists_items = tasklists_response["items"]
        return [TaskList(**item) for item in tasklists_items]

    def create_task_with_notes(self, tasklist_id: TaskListIds, title: str, notes: str, due_date: Optional[datetime] = None) -> Task:
        """Creates a new task in the specified task list with notes."""
        due_date = due_date or get_end_of_day()

        task_body = self._create_task_body(title, notes, due_date)
        task = (
            self.service.tasks().insert(tasklist=tasklist_id.value, body=task_body).execute()
        )
        return Task(**task)

    def _create_task_body(self, title: str, notes: str, due_date: datetime) -> dict:
        task_body = {
            "title": title,
            "notes": notes,
            "due": due_date.isoformat(),
        }
        
        return task_body

NameError: name 'Task' is not defined

In [8]:
service = TaskService(credentials)
tasklists = service.fetch_tasklists()

In [9]:
service.create_task_with_notes(
    TaskListIds.Default,
    "Sample Task Title",
    "This is a sample task created via the Google Tasks API.",
)

{'kind': 'tasks#task',
 'id': 'c0N1bUpzcjY0Q3ZaR21LaQ',
 'etag': '"MCTjx6U79pE"',
 'title': 'Sample Task Title',
 'updated': '2025-11-02T20:10:48.498Z',
 'selfLink': 'https://www.googleapis.com/tasks/v1/lists/MDY0Mzc2NjgyMDc4MTc0Nzg3Mjk6MDow/tasks/c0N1bUpzcjY0Q3ZaR21LaQ',
 'position': '00000000000000000000',
 'notes': 'This is a sample task created via the Google Tasks API.',
 'status': 'needsAction',
 'due': '2025-11-02T00:00:00.000Z',
 'links': [],
 'webViewLink': 'https://tasks.google.com/task/sCumJsr64CvZGmKi?sa=6'}